** valencia bounding box  
{"type":"Polygon","coordinates":[[[-0.124146,38.953446],[-0.547119,38.953446],[-0.547119,39.481696],[-0.124146,39.481696],[-0.124146,38.953446]]]}  

** data   
pre-event = 2024-10-07  
post-event = 2024-10-31  

** Data download from ASF search portal: parameters  
    * GRD-HD, ascending, instrument IW, VV+VH pols

In [ ]:
# Importing functions from package--
from pysarflow import read_grd_product

In [2]:
# Path to the .SAFE directory
product_path = (
    "../data/S1B_IW_GRDH_1SDV_20211223T051122_20211223T051147_030148_039993_5371.SAFE.zip"
)

# reading GRD product
read_grd_product(product_path)

Reading SAR product from: ../data/S1B_IW_GRDH_1SDV_20211223T051122_20211223T051147_030148_039993_5371.SAFE.zip...


3018 [main] INFO hdf.hdflib.HDFLibrary - HDF4 library: 
3018 [main] INFO hdf.hdflib.HDFLibrary -  successfully loaded.
3024 [main] INFO hdf.hdf5lib.H5 - HDF5 library: 
3025 [main] INFO hdf.hdf5lib.H5 -  successfully loaded.
INFO: eu.esa.sar.commons.io.ImageIOFile: Using FileCacheImageInputStream


	Product read successfully.


org.esa.snap.core.datamodel.Product(objectRef=0x7fb73dbafd0a)